# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.DEBUG
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [5]:
def calculate_mt(leptons, photons, jets, met):
    # Particles
    if len(leptons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    
    # Transverse mass and Delta
    cos_delta_phi = np.cos(l.phi() - met.phi())
    mt = (2 * l.pt * met.pt * (1. - cos_delta_phi))**0.5
    
    return mt

In [6]:
def calculate_phi(leptons, photons, jets, met, eta_solution=0):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = calculate_mt(leptons, photons, jets, met)
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Select one of them
        if eta_solution > 0:
            eta_v = eta_v_plus
        elif eta_solution < 0:
            eta_v = eta_v_minus
        else:
            dice = np.random.rand()
            if dice > 0.5:
                eta_v = eta_v_plus
            else:
                eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv # vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l_.vector.dot(x_)
    ly_ = l_.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    
    
def calculate_phi_minus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=-1)
    
    
def calculate_phi_plus(leptons, photons, jets, met):
    return calculate_phi(leptons, photons, jets, met, eta_solution=1)

In [7]:
def setup_observables(delphesprocessor):
    delphesprocessor.reset_observables()
    
    # Default observables (four-momenta of a, l, j, and met)
    delphesprocessor.add_default_observables(
        n_leptons_max=1,
        n_photons_max=1,
        n_jets_max=1,
        include_charge=False,
        include_numbers=False,
        include_met=True
    )
    # Lepton flavour
    delphesprocessor.add_observable('pdgid_l1', 'l[0].pdgid', required=True)

    # Two-particle systems
    delphesprocessor.add_observable('m_la', '(l[0] + a[0]).m', required=True)
    delphesprocessor.add_observable('m_lmet', '(l[0] + met).m', required=True)
    delphesprocessor.add_observable('m_amet', '(a[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_la','(l[0] + a[0]).pt', required=True)
    delphesprocessor.add_observable('pt_lmet','(l[0] + met).pt', required=True)
    delphesprocessor.add_observable('pt_amet','(a[0] + met).pt', required=True)
    delphesprocessor.add_observable('deltaphi_la', 'l[0].phi() - a[0].phi()', required=True)
    delphesprocessor.add_observable('deltaphi_lmet', 'l[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaphi_amet', 'a[0].phi() - met.phi()', required=True)
    delphesprocessor.add_observable('deltaeta_la', 'l[0].eta - a[0].eta', required=True)

    # Three-particle system
    delphesprocessor.add_observable('m_almet', '(a[0] + l[0] + met).m', required=True)
    delphesprocessor.add_observable('pt_almet', '(a[0] + l[0] + met).pt', required=True)

    # mT(W) and ressurrection phi
    delphesprocessor.add_observable_from_function('mt', calculate_mt, required=True)
    delphesprocessor.add_observable_from_function('phi_minus', calculate_phi_minus, required=True)
    delphesprocessor.add_observable_from_function('phi_plus', calculate_phi_plus, required=True)
    delphesprocessor.add_observable_from_function('phi', calculate_phi, required=True)


In [8]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.reset_cuts()
    
    if use_tight_cuts:
        delphesprocessor.add_cut('pt_a1 >= 300.')
        delphesprocessor.add_cut('pt_l1 >= 80.')
        delphesprocessor.add_cut('et_miss >= 80.')
        delphesprocessor.add_cut('(deltaphi_la**2 + deltaeta_la**2)**0.5 >= 3.')
        delphesprocessor.add_cut('eta_l1**2 < 2.4**2')
    else:
        delphesprocessor.add_cut('pt_a1 >= 20.')
        delphesprocessor.add_cut('pt_l1 >= 20.')
        delphesprocessor.add_cut('et_miss >= 20.')

## Main loop

In [9]:
n_runs_per_benchmark = 30  # Number of run_cards
n_oversampling = 3

In [11]:
run = 0

for i_card in range(n_runs_per_benchmark):
    if i_card < 1:
        continue
        
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    run = i_card + 1
    run_str = str(run)
    if len(run_str) < 2:
        run_str = '0' + run_str
    event_folder = mg_process_dir + 'Events/run_{}'.format(run_str)
    
    dp.add_sample(
        event_folder + '/tag_1_pythia8_events.hepmc.gz',
        delphes_filename=event_folder + '/tag_1_pythia8_events_delphes.root',
        lhe_filename=event_folder + '/unweighted_events.lhe.gz',
        sampled_from_benchmark='sm',
        weights="lhe",
    )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_file=log_dir + '/delphes_{}.log'.format(run - 1),
        initial_command='source activate python2',
    )
    
    # Tight analysis
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))
    
    # Loose analysis
    setup_observables(dp)
    setup_cuts(dp)
    
    dp.analyse_delphes_samples(delete_delphes_files=False)
    
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


14:35 root                 INFO    Starting analysis of runs for card 1
14:35 madminer.utils.inter INFO    HDF5 file does not contain is_reference field.
14:35 madminer.delphes     DEBUG   Adding event sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma_sys/Events/run_02/tag_1_pythia8_events.hepmc.gz
14:35 madminer.utils.inter DEBUG   Parsing HepMC line: N 1 "0" 

14:35 madminer.utils.inter DEBUG   Found weight labels in HepMC file: ['sm']
14:35 madminer.delphes     DEBUG   Delphes already run for event sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma_sys/Events/run_02/tag_1_pythia8_events.hepmc.gz
14:35 madminer.delphes     DEBUG   Resetting observables
14:35 madminer.delphes     DEBUG   Adding default observables
14:35 madminer.delphes     DEBUG   Adding required observable et_miss = met.pt
14:35 madminer.delphes     DEBUG   Adding required observable phi_miss = met.phi()
14:35 madminer.delphes     DEBUG

14:36 madminer.utils.inter DEBUG     First 10 values for observable phi_miss:
[-0.5802393   0.5238719   2.74427581 -2.28302264 -1.74556243 -0.40931362
 -0.63245082 -3.05253386 -1.04152226  1.71683609]
14:36 madminer.utils.inter DEBUG     First 10 values for observable e_visible:
[ 103.82257493  362.08208085    0.          325.08385119   48.27333206
  983.20946383  278.18077216  416.15811615 1343.75261104  356.53499344]
14:36 madminer.utils.inter DEBUG     First 10 values for observable eta_visible:
[-1.23397219e+00 -1.34403181e+00 -1.00000000e+11  9.74820614e-01
  9.90957379e-01  2.20223665e+00 -1.97666818e+00 -6.53592544e-01
 -3.76373806e+00 -1.37775349e+00]
14:36 madminer.utils.inter DEBUG     First 10 values for observable e_l1:
[ 51.91128746 181.04104042   0.         162.5419256   24.13666603
 491.60473192   0.         104.76741363 474.00313215 115.02166186]
14:37 madminer.utils.inter DEBUG     First 10 values for observable pt_l1:
[ 27.86450005  88.41482544   0.         107.360954

14:39 madminer.utils.inter DEBUG     54897 / 100000 events pass required observable phi_plus
14:39 madminer.utils.inter DEBUG     54897 / 100000 events pass required observable phi
14:39 madminer.utils.inter DEBUG     44 / 100000 events pass cut pt_a1 >= 300.
14:39 madminer.utils.inter DEBUG     24848 / 100000 events pass cut pt_l1 >= 80.
14:39 madminer.utils.inter DEBUG     16171 / 100000 events pass cut et_miss >= 80.
14:39 madminer.utils.inter DEBUG     40112 / 100000 events pass cut (deltaphi_la**2 + deltaeta_la**2)**0.5 >= 3.
14:39 madminer.utils.inter DEBUG     99309 / 100000 events pass cut eta_l1**2 < 2.4**2
14:39 madminer.utils.inter INFO      9 / 100000 events pass everything
14:39 madminer.delphes     DEBUG   Found weights ['sm'] in Delphes file
14:39 madminer.delphes     DEBUG   Found 9 events
14:39 madminer.delphes     DEBUG   Extracting weights from LHE file
14:39 madminer.delphes     DEBUG   Found weights ['sm', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '1

14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 73
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 74
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 75
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 76
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 77
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 78
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 79
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 8
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 80
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 81
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 82
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 83
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 84
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 85
14:39 madminer.utils.inter DEBUG   Adding nuisance benchmark 86
14:39 madminer.utils.inter DEBUG   Adding

14:39 madminer.delphes     DEBUG   Adding required observable deltaphi_lmet = l[0].phi() - met.phi()
14:39 madminer.delphes     DEBUG   Adding required observable deltaphi_amet = a[0].phi() - met.phi()
14:39 madminer.delphes     DEBUG   Adding required observable deltaeta_la = l[0].eta - a[0].eta
14:39 madminer.delphes     DEBUG   Adding required observable m_almet = (a[0] + l[0] + met).m
14:39 madminer.delphes     DEBUG   Adding required observable pt_almet = (a[0] + l[0] + met).pt
14:39 madminer.delphes     DEBUG   Adding required observable mt defined through external function
14:39 madminer.delphes     DEBUG   Adding required observable phi_minus defined through external function
14:39 madminer.delphes     DEBUG   Adding required observable phi_plus defined through external function
14:39 madminer.delphes     DEBUG   Adding required observable phi defined through external function
14:39 madminer.delphes     DEBUG   Resetting cuts
14:39 madminer.delphes     DEBUG   Adding cut pt_a1 

...

20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 27
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 28
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 29
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 3
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 30
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 31
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 32
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 33
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 34
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 35
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 36
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 37
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 38
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 39
20:52 madminer.utils.inter DEBUG   Adding nuisance benchmark 4
20:52 madminer.utils.inter DEBUG   Adding 

20:52 madminer.utils.inter DEBUG   Sorted benchmarks: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97']
20:52 madminer.delphes     DEBUG   Resetting observables
20:52 madminer.delphes     DEBUG   Adding default observables
20:52 madminer.delphes     DEBUG   Adding required observable et_miss = met.pt
20:52 madminer.delphes     DEBUG 

20:53 madminer.utils.inter DEBUG     First 10 values for observable phi_miss:
[ 1.5962615  -0.51825064  1.79962265 -0.57370937  0.32222846 -2.58063459
 -1.74343157  0.6196723   0.68147212  2.10793591]
20:53 madminer.utils.inter DEBUG     First 10 values for observable e_visible:
[ 6119.07501242  9689.12053591   313.01795723  2609.15257997
   362.07774242  6368.58737652  1839.3898374   1203.46489945
  2901.69938107 10707.62821498]
20:53 madminer.utils.inter DEBUG     First 10 values for observable eta_visible:
[ 0.29123596  1.13740492  0.25713299 -0.11209071 -0.42959264  1.1240139
 -0.08152888 -0.01977566  0.31754474  1.20920012]
20:53 madminer.utils.inter DEBUG     First 10 values for observable e_l1:
[2644.97544525 3224.57812477    0.         1258.80217507    0.
 1457.49732832  810.16360016  601.73244972  641.91824912 3913.81010891]
20:53 madminer.utils.inter DEBUG     First 10 values for observable pt_l1:
[2480.91650391 2826.86767578    0.         1247.9901123     0.
 1266.61352539  

20:56 madminer.utils.inter DEBUG     68997 / 100000 events pass required observable mt
20:56 madminer.utils.inter DEBUG     63871 / 100000 events pass required observable phi_minus
20:56 madminer.utils.inter DEBUG     63871 / 100000 events pass required observable phi_plus
20:56 madminer.utils.inter DEBUG     63871 / 100000 events pass required observable phi
20:56 madminer.utils.inter DEBUG     93595 / 100000 events pass cut pt_a1 >= 20.
20:56 madminer.utils.inter DEBUG     68865 / 100000 events pass cut pt_l1 >= 20.
20:56 madminer.utils.inter DEBUG     99547 / 100000 events pass cut et_miss >= 20.
20:56 madminer.utils.inter INFO      63295 / 100000 events pass everything
20:56 madminer.delphes     DEBUG   Found weights ['sm'] in Delphes file
20:56 madminer.delphes     DEBUG   Found 63295 events
20:56 madminer.delphes     DEBUG   Extracting weights from LHE file
20:56 madminer.delphes     DEBUG   Found weights ['sm', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',

20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 73
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 74
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 75
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 76
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 77
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 78
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 79
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 8
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 80
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 81
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 82
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 83
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 84
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 85
20:56 madminer.utils.inter DEBUG   Adding nuisance benchmark 86
20:56 madminer.utils.inter DEBUG   Adding

## Combine samples

In [12]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5', k_factors=1./float(n_oversampling))

22:22 madminer.sampling    DEBUG   Combining and shuffling samples
22:22 madminer.sampling    WARNING Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
22:22 madminer.sampling    INFO    Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples.h5
22:22 madminer.sampling    INFO    Loading samples from file 1 / 30 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_0.h5, multiplying weights with k factor 0.3333333333333333
22:22 madminer.sampling    INFO    Loading samples from file 2 / 30 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_1

In [13]:
tight_runs = list(range(2,10)) + list(range(12,20)) +list(range(22,30))
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in tight_runs]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5', k_factors=1./float(n_oversampling))

22:23 madminer.sampling    DEBUG   Combining and shuffling samples
22:23 madminer.sampling    WARNING Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
22:23 madminer.sampling    INFO    Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_tight_2.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_tight.h5
22:23 madminer.sampling    INFO    Loading samples from file 1 / 24 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples_tight_2.h5, multiplying weights with k factor 0.3333333333333333
22:23 madminer.sampling    INFO    Loading samples from file 2 / 24 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wg